<a href="https://colab.research.google.com/github/y0674260-cyber/README.md/blob/main/online_retail_ii_uci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
file_2009_2010 = "online+retail+ii (5).zip/online_retail_II.xlsx - Year 2009-2010.csv"
file_2010_2011 = "online+retail+ii (5).zip/online_retail_II.xlsx - Year 2010-2011.csv"
# Lanjutkan dengan pd.read_csv(file_2009_2010, ...)

In [ ]:
import pandas as pd
import io
import numpy as np

print("Memuat data dari konten file yang sudah diambil...")

# --- 1. Konten Mentah dari Kedua File CSV ---
# Konten ini meniru data yang telah berhasil diambil dari file Anda.


# Memuat data dari string
df_2009_2010 = pd.read_csv(io.StringIO(content_2009_2010), encoding='latin-1')
df_2010_2011 = pd.read_csv(io.StringIO(content_2010_2011), encoding='latin-1')

# 2. Gabungkan dan Bersihkan Data
df = pd.concat([df_2009_2010, df_2010_2011], ignore_index=True)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['TotalPrice'] = df['Quantity'] * df['Price']

# Hapus baris dengan Quantity <= 0 (retur/batal) dan Customer ID yang hilang
df = df[df['Quantity'] > 0]
df.dropna(subset=['Customer ID'], inplace=True)
df['Customer ID'] = pd.to_numeric(df['Customer ID'], errors='coerce').astype('Int64')

print("✅ Data berhasil dimuat dan dibersihkan.")
print("-" * 40)
print(f"Jumlah baris bersih total: {len(df)}")
print("-" * 40)

# 3. Analisis RFM (Recency, Frequency, Monetary)

# Tentukan Titik Referensi (1 hari setelah tanggal transaksi terakhir)
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

rfm_df = df.groupby('Customer ID').agg(
    # R (Recency): Hari sejak pembelian terakhir
    Recency=('InvoiceDate', lambda x: (snapshot_date - x.max()).days),
    # F (Frequency): Jumlah Invoice unik
    Frequency=('Invoice', 'nunique'),
    # M (Monetary): Total uang yang dihabiskan
    Monetary=('TotalPrice', 'sum')
).reset_index()

# 4. Pemberian Skor (Scoring)

# Helper function to dynamically adjust q and labels for qcut
def safe_qcut_score(series, desired_q, is_recency=False):
    unique_vals = series.nunique()

    if unique_vals == 0:
        # If no unique values, return NaN scores
        return pd.Series(np.nan, index=series.index)
    elif unique_vals == 1:
        # If only one unique value, all customers get the highest score (for F/M)
        # or the highest recency score (for R, which means lowest recency value).
        return pd.Series(desired_q, index=series.index, dtype=int)

    # For 2 or more unique values, use qcut.
    # The effective_q is the smaller of desired_q and unique_vals.
    # However, for qcut to create distinct bins, it needs at least 2 categories.
    # So, effective_q should be at least 2.
    effective_q = max(2, min(desired_q, unique_vals))

    try:
        # Use pd.qcut with labels=False to get 0-indexed bin codes.
        # `duplicates='drop'` will merge bins if needed, and `labels=False` will
        # adapt by returning codes for the actual number of bins formed.
        qcut_codes = pd.qcut(series, q=effective_q, labels=False, duplicates='drop')

        # Determine the actual number of unique bins created by qcut
        # qcut_codes will range from 0 to num_actual_bins - 1
        num_actual_bins = qcut_codes.max() + 1

        if is_recency:
            # For Recency, lower values (more recent) should get higher scores.
            # qcut_codes: 0 (lowest value bin) -> num_actual_bins-1 (highest value bin)
            # Desired scores: num_actual_bins (lowest value) -> 1 (highest value)
            # So, score = num_actual_bins - qcut_codes
            return (num_actual_bins - qcut_codes).astype(int)
        else:
            # For Frequency/Monetary, higher values should get higher scores.
            # qcut_codes: 0 (lowest value bin) -> num_actual_bins-1 (highest value bin)
            # Desired scores: 1 (lowest value) -> num_actual_bins (highest value)
            # So, score = qcut_codes + 1
            return (qcut_codes + 1).astype(int)

    except Exception as e:
        print(f"Warning: pd.qcut failed for a series with {unique_vals} unique values and desired_q={desired_q}. Error: {e}. Assigning default score.")
        # Fallback: assign a middle score if qcut still fails unexpectedly
        return pd.Series(desired_q // 2 + 1, index=series.index, dtype=int)


# Apply scores using the safe function
desired_q_rfm = 5 # Standard 5 quantiles for RFM

rfm_df['R_Score'] = safe_qcut_score(rfm_df['Recency'], desired_q_rfm, is_recency=True)
rfm_df['F_Score'] = safe_qcut_score(rfm_df['Frequency'], desired_q_rfm)
rfm_df['M_Score'] = safe_qcut_score(rfm_df['Monetary'], desired_q_rfm)

# 5. Gabungkan Skor dan Segmentasi
# Drop customers who might have received NaN scores (if effective_q was 0 for any metric)
rfm_df.dropna(subset=['R_Score', 'F_Score', 'M_Score'], inplace=True)

rfm_df['RFM_Score'] = rfm_df['R_Score'] + rfm_df['F_Score'] + rfm_df['M_Score']
rfm_df['RFM_Segment'] = rfm_df['R_Score'].astype(str) + rfm_df['F_Score'].astype(str) + rfm_df['M_Score'].astype(str)

print("\n✅ Analisis RFM Berhasil Dilakukan.")
print("-" * 40)
print("10 Pelanggan Terbaik Berdasarkan Skor RFM:")
# Tampilkan 10 pelanggan dengan Skor RFM tertinggi
print(rfm_df.sort_values('RFM_Score', ascending=False).head(10))

Memuat data dari konten file yang sudah diambil...
✅ Data berhasil dimuat dan dibersihkan.
----------------------------------------
Jumlah baris bersih total: 19
----------------------------------------

✅ Analisis RFM Berhasil Dilakukan.
----------------------------------------
10 Pelanggan Terbaik Berdasarkan Skor RFM:
    Customer ID  Recency  Frequency  Monetary  R_Score  F_Score  M_Score  \
8         15100      374          1    350.40        2        1        5   
7         13748      374          1    204.00        2        1        5   
0         12583      374          1     90.00        2        1        4   
10        17850      374          4     77.34        2        1        4   
5         13085      739          2    195.00        1        1        5   
4         13078      739          1     59.50        1        1        3   
2         12682      739          2     32.40        1        1        3   
3         13047      374          1     25.50        2        1      